In [ ]:
!pip install transformers -U
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import os
import pickle
import concurrent.futures
from dotenv import load_dotenv
import shutil
import zipfile

load_dotenv()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model= BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/CustomModel/CustomModel")

In [ ]:
def extract_page_content(pikl_files):
    all_page_content = []
    for pikl_file in pikl_files:
        with open(pikl_file, 'rb') as file:
            pdf_chunks = pickle.load(file)

        for pdf_chunk in pdf_chunks:
            page_content = pdf_chunk.get('page_content')
            if page_content is not None:
                all_page_content.append(page_content)

    return all_page_content

In [ ]:
def get_pdf_pikl_files(root_directory):
    pikl_files = []
    for root, dirs, files in os.walk(root_directory):
        for file in files:
            if file.lower().endswith('.pkl'):
                pikl_files.append(os.path.join(root, file))
    return pikl_files

In [ ]:
def extract_to_original_pickle(zip_file, destination_folder):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(destination_folder)

In [ ]:
# if __name__ == "__main__":
zip_file_path = '/content/drive/MyDrive/intern_fastai/set4_chunks.zip'
original_pickle_folder = './original_pickle'

if os.path.exists(zip_file_path):
    # Extract all pickle files to the "original_pickle" folder
    if not os.path.exists(original_pickle_folder):
        os.makedirs(original_pickle_folder)
    extract_to_original_pickle(zip_file_path, original_pickle_folder)
    print("All pickle files extracted to 'original_pickle' folder.")

All pickle files extracted to 'original_pickle' folder.


In [ ]:
import csv
if __name__ == "__main__":

    pikl_folder = '/content/original_pickle'
    output_csv_file = 'output.csv'


    if os.path.exists(pikl_folder):
        pikl_files = get_pdf_pikl_files(pikl_folder)
        all_page_content = extract_page_content(pikl_files)

        # Create a DataFrame with the collected page content
        df = pd.DataFrame({'page_content': all_page_content})

        # Save the DataFrame to a CSV file
        # df.head()
        # df.to_csv(output_csv_file, index=False)
        df.to_csv(output_csv_file, index=False, quoting=csv.QUOTE_ALL, escapechar='\\')


        print(f"Data extracted and stored in {output_csv_file}.")


Data extracted and stored in output.csv.


In [ ]:
import os
import pandas as pd
from transformers import BertTokenizer

def preprocess_text(text_data, tokenizer):
    preprocessed_data = tokenizer(text_data, truncation=True, max_length=128, padding='max_length', return_tensors='pt')
    return preprocessed_data

def main():
    # csv_file = input("/content/output_2_chunks.csv")

    # Load the CSV file into a pandas DataFrame

    df = pd.read_csv('/content/output.csv')

    # Initialize the tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Preprocess the text data
    valid_indices = []
    for index, row in df.iterrows():
        text_data = row['page_content']

        try:
            preprocessed_data = preprocess_text(text_data, tokenizer)
            valid_indices.append(index)
        except Exception as e:
            print(f"Invalid entry found at index {index}: {e}")

    # Keep only the valid entries in the DataFrame
    df = df.loc[valid_indices]

    # Save the cleaned DataFrame back to the CSV file
    df.to_csv("refined.csv", index=False)

    print("CSV file cleaned successfully.")

if __name__ == "__main__":
    main()


Invalid entry found at index 2750: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).
Invalid entry found at index 3682: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).
Invalid entry found at index 17427: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).
Invalid entry found at index 17451: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).
Invalid entry found at index 17487: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).
Invalid entry found at index 17734: text input must 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:


# Read the CSV file with 'chunkText' column
csv_file = '/content/refined.csv'
df = pd.read_csv(csv_file)

# Batch size for processing (change as per your requirement)
batch_size = 250

# Initialize lists to store predictions for all rows
all_predictions = []

# Iterate through the DataFrame in batches of size 'batch_size'
for start_idx in range(0, len(df), batch_size):
    end_idx = min(start_idx + batch_size, len(df))
    df_batch = df.iloc[start_idx:end_idx]

    # Extract the 'chunkText' column from the current batch
    text_data = df_batch['page_content'].tolist()

    # Preprocess the text data using the tokenizer
    max_length = tokenizer.model_max_length
    preprocessed_data = tokenizer(text_data, truncation=True, max_length=max_length, padding='max_length', return_tensors='pt')

    with torch.no_grad():
        model.eval()  # Set the model to evaluation mode
        inputs = {k: v.to(model.device) for k, v in preprocessed_data.items()}
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1).tolist()

    # Append predictions for the current batch to the overall list
    all_predictions.extend(predictions)

# 'all_predictions' now contains the predicted class labels for all rows in the CSV file
print(all_predictions)


[1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 

In [ ]:
a=pd.read_csv("/content/refined.csv")

In [ ]:
a['predictions']=all_predictions

In [ ]:
a.shape

(38892, 2)

In [ ]:
a.to_csv("set_4.predictions.csv",index=False)

In [ ]:
data=pd.read_csv("/content/output.csv")
data.head()

,page_content
0,Article\n\nSame Same but Diﬀerent: How and Why...
1,"Sustainability 2019, 11, 2267; doi:10.3390/su1..."
2,The research contributes to theory and practic...
3,The course of action constitutes the strategy ...
4,"Sustainability 2019, 11, x FOR PEER REVIEW 3 o..."
